<a href="https://colab.research.google.com/github/Manvi190502/AgenticAI/blob/main/using_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade langchain langchain-community langgraph wikipedia google-genai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.7/724.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.4 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=8ea58ea39e14e65d4a225bdf13dc90deca1377bd1fa11777c0bc5cfbff6f2526
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac

In [14]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.9 MB/s eta 0:00:00


In [30]:
from google.colab import userdata
from langchain_groq import ChatGroq

api_key = userdata.get("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    groq_api_key=api_key,
    temperature=0
)

In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [17]:
wiki_tool.run({"query":"AI agents"})

'Page: AI agent\nSummary: In the context of generative artificial intelligence, AI agents (also referred to as compound AI systems or agentic AI) are a class of intelligent agents distinguished by their ability to operate autonomously in complex environments. Agentic AI tools prioritize decision-makin'

In [20]:
tools=[wiki_tool]
llm_with_tools= llm.bind_tools(tools)
result = llm_with_tools.invoke("Hello world!")
result
result.content

"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [36]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm,tools)

/tmp/ipython-input-3351805951.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm,tools)


In [32]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='a5921c19-2c02-4402-8afc-df7433b22782'),
 AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 283, 'total_tokens': 308, 'completion_time': 0.073918883, 'completion_tokens_details': None, 'prompt_time': 0.01378406, 'prompt_tokens_details': None, 'queue_time': 0.171553916, 'total_time': 0.087702943}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_45180df409', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6512-2707-7ad2-ac7c-8348d8d0e4f4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 283, 'output_tokens': 25, 'total_tokens': 308})]

In [23]:
print(response["messages"][-1].content)

Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [37]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke({
    "messages": [HumanMessage(content="What is agentic ai?")]
})

print(response["messages"][-1].content)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=wikipedia{"query": "Agentic AI"}</function>'}}

In [52]:
from google.colab import userdata
from google import genai

# Get API key from Colab Secrets
api_key = userdata.get("GOOGLE_API_KEY")

# Create client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is agentic AI?"
)

print(response.text)


**Agentic AI** refers to a type of artificial intelligence system designed to operate with a high degree of **autonomy**, **initiative**, and **goal-directed behavior**. Unlike a simple AI model that reacts to specific prompts or performs a single task, an agentic AI is capable of perceiving its environment, setting its own sub-goals, planning multi-step actions, executing those actions, and adapting based on feedback, all with the ultimate aim of achieving a broader, often complex, objective.

Think of it less as a tool and more as a digital *worker* or *assistant* that can reason, plan, and take action independently.

### Key Characteristics of Agentic AI:

1.  **Autonomy:** It can operate without constant human intervention, making its own decisions about how to proceed.
2.  **Goal-Oriented:** It has a clearly defined objective or mission it's trying to achieve. This goal might be high-level (e.g., "research the market for renewable energy stocks") and the agent will break it down i

In [66]:
# ---- Imports ----
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# ---- Load API Key ----
api_key = userdata.get("GOOGLE_API_KEY")

# ---- Initialize Gemini (use the working model) ----
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",   # IMPORTANT: use the model that worked
    google_api_key=api_key,
    temperature=0
)

# ---- Wikipedia Tool ----
api_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=500
)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki_tool.description = (
    "Use this tool ALWAYS to search Wikipedia for answering any question."
)

# ---- Create Agent ----
agent = create_agent(
    model=llm,
    tools=[wiki_tool],
    system_prompt="Always use the Wikipedia tool to answer questions."
)

# ---- Invoke Agent ----
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "Search Wikipedia for agentic AI and summarize it."}
    ]
})

print(response["messages"][-1])


content=[{'type': 'text', 'text': 'AI agents, also known as compound AI systems or agentic AI, are a type of intelligent agent in generative artificial intelligence. They are characterized by their ability to function independently in intricate environments. These tools focus on decision-making rather than content creation and do not need constant supervision.', 'extras': {'signature': 'CvcBAb4+9vuPl36TEgOmJMQDv4R+iBVHLvyyg9RJEUdnAoaL/yFIAZ/j5tN5PHfjESO5WYmYf473PEiUUjXQBujAptd9LfF1ExdY1kA92pNmE6yy0O8D34kCEEkHlkj6xRvLIHLe2FeOKfE+yj8bNss2hpef45g94yATG6ufl+cYWYaU2fGpz7iItIY92pCBChG4LvT5eA2l5T4MdS0PNaPUPjLPCTQELG8X1o+hmrcxMr//KSrNXEPmh5NEbFRmV0GwXS0EFt21YFKKkCQzvwc6NXiHB4kT7DJ1s2EdJJectC/KtFcP4LfKtScJ7kN2bdBt3C5HayZfcg=='}}] additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c6577-8543-7483-bdf9-2aa29188f5f4-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tok

In [59]:
print(response["messages"][-1].content[0]["text"])

Agentic AI, also known as AI agents or compound AI systems, refers to a class of intelligent agents that can operate autonomously in complex environments. They are characterized by their ability to prioritize decision-making over content creation and do not require continuous oversight.


In [50]:
for model in client.models.list():
    print(model.name)


models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025
models/gemini-embedding-001
models/aqa
models/imagen-4.0-generate-preview-06-06
models/imagen